<a href="https://colab.research.google.com/github/rjhanjee/my-first-repo/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import sqlite3 as sq3
import numpy as np

# Download the CSV files
!wget https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/county_pop_arcos.csv
!wget https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/land_area.csv
!wget https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/county_annual.csv


# Create a connection to the database
con = sq3.connect("opioid.db")

# Read the CSV files into pandas DataFrames
population_df = pd.read_csv('county_pop_arcos.csv')
annual_df = pd.read_csv('county_annual.csv')
land_df = pd.read_csv('land_area.csv')

# Write the DataFrames to tables in the database
# if_exists='replace' will overwrite the table if it already exists
population_df.to_sql('population', con, if_exists='replace', index=False)
annual_df.to_sql('annual', con, if_exists='replace', index=False)
land_df.to_sql('land', con, if_exists='replace', index=False)

# Close the connection
con.close()

print("CSV files imported into opioid.db successfully.")

# print few rows of the population data

print("Here are few rows of population data")

print(population_df[['BUYER_COUNTY', 'BUYER_STATE','STATE','COUNTY','year','population']].head())

print("- " *30)

# Now print some data in the annual dataset

print("now print some data in the annual dataset")

annual_df = annual_df.drop(['Unnamed: 0'],\
             axis = 1)


nan_rows = annual_df[annual_df['countyfips'].isna()]
print(annual_df[annual_df.countyfips.isna() == True].iloc[:10,:])

print("- " *30)

print(annual_df[(annual_df.countyfips.isna() == True) & (annual_df.BUYER_STATE != "PR")].iloc[:10,:])

print("- " *30)

# update annual set countyfips = 05097 where BUYER_STATE = "AR" and BUYER_COUNTY = "MONTGOMERY"
annual_df.loc[(annual_df['BUYER_STATE'] == 'AR') & (annual_df['BUYER_COUNTY'] == 'MONTGOMERY'), 'countyfips'] = 5097
print(annual_df[(annual_df.BUYER_STATE == "AR") & (annual_df.BUYER_COUNTY == "MONTGOMERY")].iloc[:10,:])
#

# delete from annual where BUYER_COUNTY = "NaN"
annual_df = annual_df.dropna(subset=['BUYER_COUNTY'])
print(annual_df[annual_df.BUYER_COUNTY.isna() == True].iloc[:10,:])

#create table land_area as select Areaname, STCOU, LND110210D from land;
# we want to grab just three columns from the land.df table, so let’s create a new one called land_area
land_area = land_df[['Areaname', 'STCOU', 'LND110210D']]
land_area.columns = ['Areaname', 'STCOU', 'land_area']

print("Table land_area")
print()
print(land_area.head())
print("-"*30)

# Also, the column there is called STCOU, which we want to rename to coutyfips
land_area.rename(columns={'STCOU': 'countyfips'}, inplace=True)

print("Table land_area")
print()

print(land_area.head())

print("-"*30)

#  left join our table and print out the counts to make sure we accounted correctly.

# create a table county_info as select * from population_df
# 2. Perform the left join using pd.merge()
# The 'on' parameter specifies the common column 'countyfips'.
# The 'how' parameter is set to 'left' to perform a left join.
county_info = pd.merge(population_df, land_area, on='countyfips', how='left')

# 3. Print the resulting 'county_info' DataFrame
print("County Info Table (DataFrame):")
print(county_info.head())

row_count_land_area = len(land_area)
row_count_population = len(population_df)
row_count_county_info = len(county_info)
print(f"Number of rows in land_area: {row_count_land_area}")
print(f"Number of rows in population_df: {row_count_population}")
print(f"Number of rows in county_info: {row_count_county_info}")

--2025-08-30 18:48:48--  https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/county_pop_arcos.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2771268 (2.6M) [text/plain]
Saving to: ‘county_pop_arcos.csv.31’

county_pop_arcos.cs 100%[===================>]   2.64M  --.-KB/s    in 0.08s   

2025-08-30 18:48:48 (35.1 MB/s) - ‘county_pop_arcos.csv.31’ saved [2771268/2771268]

--2025-08-30 18:48:48--  https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/land_area.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

/tmp/ipython-input-2233795405.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land_area.rename(columns={'STCOU': 'countyfips'}, inplace=True)
